In [ ]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 13.8 MB/s 
     |████████████████████████████████| 163 kB 63.0 MB/s 
     |████████████████████████████████| 6.6 MB 60.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 432 kB 14.4 MB/s 
     |████████████████████████████████| 212 kB 69.1 MB/s 
     |████████████████████████████████| 115 kB 57.6 MB/s 
     |████████████████████████████████| 127 kB 73.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
import pandas as pd
import numpy as np
import datasets
import transformers


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/nlp_learnground/7.10-emo_ds/emo_ds.csv", index_col=0)
df.head()

,sentence,label,label_enc
0,i didnt feel humiliated,sadness,4
1,i can go from feeling so hopeless to so damned...,sadness,4
2,im grabbing a minute to post i feel greedy wrong,anger,0
3,i am ever feeling nostalgic about the fireplac...,love,3
4,i am feeling grouchy,anger,0


In [ ]:
df['label_enc'].value_counts()

2    6761
4    5797
0    2709
1    2373
3    1641
5     719
Name: label_enc, dtype: int64

In [ ]:
df_filtered = df[['sentence', 'label_enc']]
df_filtered_two = df[['sentence', 'label']]


In [ ]:
slicer = int(len(df_filtered_two) *0.7)
df_train = df_filtered[:slicer]
df_test = df_filtered[slicer:]
print(len(df_train), len(df_test))

In [ ]:
df_train.to_csv("/content/drive/MyDrive/nlp_learnground/7.10-emo_ds/emo_ds_filtered_train.csv", index=False)
df_test.to_csv("/content/drive/MyDrive/nlp_learnground/7.10-emo_ds/emo_ds_filtered_test.csv", index=False)

# Start here: load dataset

In [ ]:
from datasets import load_dataset
#Encoded Labels dataset
dataset = load_dataset('csv', data_files={'train': '/content/drive/MyDrive/nlp_learnground/7.10-emo_ds/emo_ds_filtered_train.csv', \
                                          'test': '/content/drive/MyDrive/nlp_learnground/7.10-emo_ds/emo_ds_filtered_test.csv'})

#
# dataset = load_dataset('csv', data_files={'train': '/content/drive/MyDrive/nlp_learnground/7.10-emo_ds/emo_ds_filtered_train_label.csv', \
#                                           'test': '/content/drive/MyDrive/nlp_learnground/7.10-emo_ds/emo_ds_filtered_test_label.csv'})

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset #text, label

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 14000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 6000
    })
})

In [ ]:
dataset['train'][0]

{'text': 'i didnt feel humiliated', 'label': 4}

In [ ]:
# SECOND ATTEMPT 

from transformers import TrainingArguments, Trainer

import numpy as np
from datasets import load_metric

from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from transformers import TrainingArguments, Trainer

metric = load_metric("accuracy")
 
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
def tokenize_data(example):
    return tokenizer(example['text'], padding='max_length')

dataset_trans = dataset.map(tokenize_data, batched=True)
#dataset_trans = dataset_trans.map(remove_columns='sentence')

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=6)

training_args = TrainingArguments(output_dir="test_trainer", \
                                  evaluation_strategy="epoch", \
                                  num_train_epochs=3, \
                                  )        #(0.001) per_device_train_batch_size=16, \ learning_rate=1e-3

trainer = Trainer(
    model=model, args=training_args, train_dataset= dataset_trans['train'], eval_dataset= dataset_trans['test'], compute_metrics=compute_metrics
)

trainer.train()

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/a8d257ba9925ef39f3036bfc338acf5283c512d9/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.22.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/a8d257ba9925ef39f3036bfc338acf5283c512d9/vocab.txt
loading file tokenize

Epoch,Training Loss,Validation Loss,Accuracy
1,0.307800,0.254686,0.928833
2,0.163000,0.196860,0.931500


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json
Model weights saved in test_trainer/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1500
Configuration saved in test_trainer/checkpoint-1500/config.json
Model weights saved in test_trainer/checkpoint-1500/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 6000
  Batch size = 8
Saving model checkpoint to test_trainer/checkpoint-2000
Configuration saved in test_trainer/checkpoint-20

In [ ]:
# First approach 
from transformers import TrainingArguments, Trainer

import numpy as np
from datasets import load_metric

from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from transformers import TrainingArguments, Trainer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")
 
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)
    
 
tokenized_datasets = dataset.map(tokenize_function, batched=True)


checkpoint = "distilbert-base-cased"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=6)

import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
print(device)
 
# def compute_metrics(eval_pred):
#     metric1 = load_metric("precision")
#     metric2 = load_metric("recall")
#     metric3 = load_metric("accuracy")
#     metric4 = load_metric("f1")
    
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)
#     precision = metric1.compute(predictions=predictions, references=labels)["precision"]
#     recall = metric2.compute(predictions=predictions, references=labels)["recall"]
#     accuracy = metric3.compute(predictions=predictions, references=labels)["accuracy"]
#     f1 = metric4.compute(predictions=predictions, references=labels)["f1"]
#     return {"precision": precision, "recall": recall, "accuracy":accuracy, "f1":f1}


def compute_metrics(eval_pred):
    metric = load_metric("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(output_dir="test_trainer1", evaluation_strategy="epoch", num_train_epochs=3)
 
 
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics
)
 
trainer.train()

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-cased/snapshots/8d708decd7afb7bec0af233e5338fe1fca3db705/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-cased",
  "activation": "gelu",
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.22.2",
  "vocab_size": 28996
}

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--distilbert-base-cased/snapshots/8d708decd7afb7bec0af233e5338fe1fca3db705/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-cased/snapshots/8d708decd7afb7bec0af233e5338fe1fca3

cuda


Epoch,Training Loss,Validation Loss,Accuracy
1,0.301200,0.204732,0.928000
2,0.164300,0.235428,0.932333
3,0.109400,0.240060,0.929833


Saving model checkpoint to test_trainer1/checkpoint-500
Configuration saved in test_trainer1/checkpoint-500/config.json
Model weights saved in test_trainer1/checkpoint-500/pytorch_model.bin
Saving model checkpoint to test_trainer1/checkpoint-1000
Configuration saved in test_trainer1/checkpoint-1000/config.json
Model weights saved in test_trainer1/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to test_trainer1/checkpoint-1500
Configuration saved in test_trainer1/checkpoint-1500/config.json
Model weights saved in test_trainer1/checkpoint-1500/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 6000
  Batch size = 8
Saving model checkpoint to test_trainer1/checkpoint-2000
Configuration saved in test

TrainOutput(global_step=5250, training_loss=0.2347898911975679, metrics={'train_runtime': 2327.8884, 'train_samples_per_second': 18.042, 'train_steps_per_second': 2.255, 'total_flos': 5564027621376000.0, 'train_loss': 0.2347898911975679, 'epoch': 3.0})

In [ ]:
! mv /content/test_trainer1/checkpoint-2500 /content/drive/MyDrive/nlp_learnground/7.10-emo_ds/checkpoint  